# 사실은 이게 6-2인데 커널 다시 돌리기 구차나서 이름은 그대로 놔두고 할럽니다

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import csv

import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px
import folium
# import networkx as nx

In [2]:
import json

with open('./geojson/new.geojson') as j:
    seoul_geo = json.load(j)

# 반복해서 **2020년 전체 12달** 뽑아보기

In [4]:
# import pandas as pd
from os import listdir

with open("../참고 데이터/한국행정구역분류_행정동코드(서울).csv", encoding='cp949') as f:
    data1 = pd.read_csv(f)
    newdata = data1[['소분류', '읍면동']]

bigdf = pd.DataFrame()

folderpaths = [
    ["../참고 데이터/csv/large-datasets/2020/"+ folder + '/' + f for f in listdir("../참고 데이터/csv/large-datasets/2020/" + folder)]
    for folder in listdir("../참고 데이터/csv/large-datasets/2020/") if not folder.endswith(".zip")
]

for monthfolder in folderpaths:
    for file in monthfolder[:]:
        with open(file, encoding='cp949') as f:
            data = pd.read_csv(f)

        data = data.drop(data[data['출발 시군구 코드'] == data['도착 시군구 코드']].index)
        # data.loc[data['이동인구(합)'] == '*', '이동인구(합)'] = 0/
        data['이동인구(합)'] = data['이동인구(합)'].replace(to_replace="*", value=0)
        data['이동인구(합)'] = data['이동인구(합)'].astype(float)

        # tst = {key: value for key, value in zip(data1['소분류'], data1['시군구']+" "+data1['읍면동'])} # 이건 자치구용
        tst = {key: value for key, value in zip(data1['중분류'], data1['시군구'])}
        #tst = {11010:"ㅁㄴㅇㄹ"}
        # print(tst)

        data = data.drop(data[data['출발 시군구 코드'].astype(int) > 11250].index)
        data = data.drop(data[data['도착 시군구 코드'].astype(int) > 11250].index)
        data = data.replace(tst)

        # print(data.dtypes)

        data2 = data.groupby(['출발 시군구 코드', '도착 시군구 코드'
                            ])['평균 이동 시간(분)'].sum()  #.sort_values()  #.head(10)
        # print(data2)

        # data['이동인구(합)'] = data[].astype(float)

        # data2point5 = pd.DataFrame(data.groupby(['도착 시군구 코드'])['평균 이동 시간(분)'].sum().sort_values())
        data2point5 = pd.DataFrame(data.groupby(
            ['도착 시군구 코드'])['이동인구(합)'].sum())  #.sort_values())
        data2point5 = data2point5.reset_index()
        # data2point5 = pd.DataFrame(data2)

        new_row = data2point5
        new_row['time'] = int(file.split("/")[6].split("_")[3][0:2])
        bigdf = pd.concat([bigdf, new_row])
    '''
        new_row = data2point5.T.iloc[1:,:] # remove upper row
        # new_row['']
        # new_row = data2point5.T
        bigdf = pd.concat([bigdf, new_row])
        # print(list(data2))
        # regions = ['금천구','종로구','중구','용산구','구로구','도봉구','강북구','중랑구','양천구','영등포구','강동구','성동구','동대문구','서대문구','광진구','은평구','마포구','동작구','강서구','노원구','성북구','서초구','관악구','강남구','송파구']
        # regions = data2point5['도착 시군구 코드'].T
        # print(list(regions))
        #regions = ['금천구','종로구','양천구','용산구','도봉구','성동구','중구','구로구','서대문구','중랑구','강북구','동대문구','광진구','동작구','은평구','서초구','영등포구','강동구','성북구','강서구','노원구','관악구','마포구','송파구','강남구']
    bigdf = bigdf.reset_index().iloc[: , 1:] # remove 이동인구(합)x25 column
    bigdf.rename(#columns={i:r for i,r in zip(range(25),regions)}, 
                index={i:str(j)+"시" for i,j in enumerate(range(24))}, inplace=True)
    bigdf = bigdf.T
    bigdf['regions'] = [
        '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구',
        '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구',
        '용산구', '은평구', '종로구', '중구', '중랑구'
    ]
    '''
    
    fig = px.choropleth_mapbox(bigdf,
                           geojson=seoul_geo,
                           locations='도착 시군구 코드',
                           color='이동인구(합)',
                           color_continuous_scale="Viridis",
                           featureidkey="properties.name",
                           mapbox_style="carto-positron",
                           opacity=0.5,
                           zoom=10,
                           center={
                               "lat": 37.559,
                               "lon": 126.963
                           },
                           labels={'이동인구(합)':'label'},
                           animation_frame='time'
                           )
    a = file.split("/")[6].split("_")[2]
    fig.write_html(f'../작업 결과물/choropleth1/시간별지도_({a}).html')

# 13분 50.7초

# 반복해서 **2020년 출발,도착 따로 만들기**

In [17]:
# import pandas as pd
import json
from os import listdir

with open("../참고 데이터/한국행정구역분류_행정동코드(서울).csv", encoding='cp949') as f:
    data1 = pd.read_csv(f)
    newdata = data1[['소분류', '읍면동']]

folderpaths = [
    # ["../참고 데이터/csv/large-datasets/2020/" + folder + '/' + f for f in listdir("../참고 데이터/csv/large-datasets/2020/" + folder)]
    "../참고 데이터/csv/large-datasets/2020/" + folder for folder in listdir("../참고 데이터/csv/large-datasets/2020/") if not folder.endswith(".zip")
]

for folder in folderpaths:
    data = pd.concat([
        pd.read_csv(folder + '/' + file,
                    encoding='cp949')
        for file in listdir(folder)
    ],
                     ignore_index=True)

    data = data.drop(data[data['출발 시군구 코드'] == data['도착 시군구 코드']].index)
    # data.loc[data['이동인구(합)'] == '*', '이동인구(합)'] = 0/
    data['이동인구(합)'] = data['이동인구(합)'].replace(to_replace="*", value=0)
    data['이동인구(합)'] = data['이동인구(합)'].astype(float)

    # tst = {key: value for key, value in zip(data1['소분류'], data1['시군구']+" "+data1['읍면동'])} # 이건 자치구용
    tst = {key: value for key, value in zip(data1['중분류'], data1['시군구'])}
    #tst = {11010:"ㅁㄴㅇㄹ"}
    # print(tst)

    data = data.drop(data[data['출발 시군구 코드'].astype(int) > 11250].index)
    data = data.drop(data[data['도착 시군구 코드'].astype(int) > 11250].index)
    data = data.replace(tst)

    # print(data.dtypes)

    data2 = data.groupby(['출발 시군구 코드', '도착 시군구 코드'])['평균 이동 시간(분)'].sum().sort_values()#.head(10)
    # print(data2)

    # data['이동인구(합)'] = data[].astype(float)

    # data2point5 = pd.DataFrame(data.groupby(['도착 시군구 코드'])['평균 이동 시간(분)'].sum().sort_values())
    data2point5 = pd.DataFrame(data.groupby(['도착 시군구 코드'])['이동인구(합)'].sum().sort_values())
    data2point5 = data2point5.reset_index()
    # data2point5 = pd.DataFrame(data2)
    # data2point5
    # print(list(data2))
    #regions = ['금천구','종로구','양천구','용산구','도봉구','성동구','중구','구로구','서대문구','중랑구','강북구','동대문구','광진구','동작구','은평구','서초구','영등포구','강동구','성북구','강서구','노원구','관악구','마포구','송파구','강남구']
    data2point6 = pd.DataFrame(data.groupby(['출발 시군구 코드'])['이동인구(합)'].sum().sort_values())
    data2point6 = data2point6.reset_index()
    # data2point5 = pd.DataFrame(data2)
    # data2point6
    a = folder.split("_")[2]

    m1 = folium.Map(
        location=[37.559819, 126.963895],
        zoom_start=11,
        tiles='cartodbpositron'
    )

    folium.Choropleth(
        geo_data=seoul_geo,
        name="choropleth",
        data=data2point5,
        columns=["도착 시군구 코드", "이동인구(합)"],
        key_on="properties.name",
        fill_color="OrRd",
        fill_opacity=0.7,
        line_opacity=.1,
        legend_name="레게노",
    ).add_to(m1)

    folium.LayerControl().add_to(m1)

    m1.save(f"../작업 결과물/choropleth2/도착/1달간+도착_({a}).html")


    m2 = folium.Map(location=[37.559819, 126.963895],
               zoom_start=11,
               tiles='cartodbpositron')

    folium.Choropleth(
        geo_data=seoul_geo,
        name="choropleth",
        data=data2point6,
        columns=["출발 시군구 코드", "이동인구(합)"],
        key_on="properties.name",
        fill_color="PuBu",
        fill_opacity=0.7,
        line_opacity=.1,
        legend_name="레게노",
    ).add_to(m2)

    folium.LayerControl().add_to(m2)

    m2.save(f"../작업 결과물/choropleth2/출발/1달간+출발_({a}).html")

# 